# Script 2 - Clean Data

_Script by Tim Hebestreit, thebestr@smail.uni-koeln.de_

In this notebook, the raw extracted articles are read in, cleaned, filtered, and saved with additional engineered features. The resulting corpus_final.csv is already present in the data/csv folder, but to replicate the process you can simply run all cells in this script. This process can take a short while, but should not be longer than 1-2 minutes.

In [1]:
# --- IMPORTS ---
import pandas as pd
import re
import os
from tqdm.notebook import tqdm

We define the input file (the raw articles exported using Script 1), and the name of the output file.

In [2]:
# --- CONFIG ---
INPUT_FILE = "../data/csv/articles_raw.csv"
OUTPUT_FILE = "../data/csv/corpus.csv"

# Enable tqdm for pandas operations
tqdm.pandas()

Here, we list the most common IT sources across different industries. This will be useful later to classify the articles as IT articles and general articles.

We also define possible Press Release sources. Articles from these sources will be filtered out, as they are likely just corporate communication instead of journalistic articles.

In [3]:
# --- DEFINE SOURCES ---

# List possible Tech sources to catch IT articles
# Allows distinction between general public media and specialized discourse

# Disclaimer: As no comprehensive list was found online, these sources were originally generated with AI
# and then validated based on which of these sources were actually present in the dataset
IT_SOURCES = [
    r'\(wiso\)', # Catch-all for WISO PDF exports
    'autoflotte',
    'autohaus',
    'automobil',
    'computerbild',
    'computerwoche',
    'egovernment',
    'elektronik',
    'elektrotechnik',
    'energie & management',
    'entsorga',
    'internet world',
    'it business',
    'ix',
    'kfz betrieb',
    'konstruktion',
    'labor praxis',
    'logistik',
    'logistra',
    'macwelt',
    'maschinenmarkt',
    'pcwelt',
    'process',
    'telecom handel',
    'transport',
    'vdi',
    'verkehrsrundschau',
]

# List is empty because preliminary checks showed no PR sources in the raw data
# Kept as variable for consistency and potential future extendability
PR_SOURCES = []

Now the raw CSV file generated by Script 1 is loaded and its shape and number of missing values per column are returned.

In [4]:
# --- LOAD AND INSPECT RAW DATA ---

if os.path.exists(INPUT_FILE):
    df = pd.read_csv(INPUT_FILE)
    print(f"Raw data loaded. Initial shape: {df.shape}")
    
    # Basic data integrity check
    print("\nMissing values per column:")
    print(df.isna().sum())
    
else:
    print(f"Input file not found: {INPUT_FILE}")
    print("Please run Script 1 first to generate the raw data.")


Raw data loaded. Initial shape: (76954, 4)

Missing values per column:
Date_Raw    3808
Source         0
Title          0
Text         132
dtype: int64


Basic cleaning is performed in the next cell by removing rows without text or a title, as well as duplicate articles. Also, excessive whitespace is filtered from the text body of each article.

In [5]:
# --- BASIC CLEANING ---

initial_count = len(df)

# Remove rows with missing text or title
df = df.dropna(subset=['Text', 'Title'])

# Text Cleanup
# Remove excessive whitespace (newlines and tabs) to save space and make text cleaner
# This is done before removing duplicates to catch duplicate articles differing only by whitespaces
df['Text'] = df['Text'].astype(str).str.replace(r'\s+', ' ', regex=True).str.strip()

# Remove Duplicates
# Nexis can export duplicates if search batches overlap, and Nexis and Wiso articles can overlap
# We assume that if title and text are identical, then it is the same article
df = df.drop_duplicates(subset=['Title', 'Text'])

cleaned_count = len(df)
print(f"Removed {initial_count - cleaned_count} duplicates/empty rows.")
print(f"Remaining articles: {cleaned_count}")

Removed 12946 duplicates/empty rows.
Remaining articles: 64008


Next, we wish to parse the date string to an actual date object. For that, we first define a helper function that parses a date string. 

In [6]:
# --- HELPER FUNCTION ---

def parse_date(raw_date):
    """Parses date strings in German into datetime objects."""
    if not isinstance(raw_date, str):
        return pd.NaT
    
    # Clean up the date string (remove 'Date:', 'Datum:', extra spaces)
    raw_date = raw_date.strip()
    
    # First look for standard German/English date string (DD. Month YYYY) with pattern:
    # 1-2 digits
    # optional dot
    # space
    # letters (for the month)
    # optional dot (for abbreviations)
    # space
    # 4 digits (Year)
    match = re.search(r'(\d{1,2})\.?\s+([a-zA-ZäöüÄÖÜ]+)\.?\s+(\d{4})', raw_date)
    if match:
        day, month, year = match.groups()
        
        # Translate the month if it is in German
        # Check bothfull name and abbreviation
        if month in german_months:
            month = german_months[month]

        # Normalize year (e.g. '20' -> '2020')
        if len(year) == 2:
            year = "20" + year
            
        
        # Tries to return a clean string, e.g. 15 October 2025, for pandas to turn into datetime object
        try:
            return pd.to_datetime(f"{day} {month} {year}", errors='coerce')
        except:
            pass # Fall through to next strategy

    # If the first regex did not match look for numeric dates (DD.MM.YYYY or DD.MM.YY)
    match_numeric = re.search(r'(\d{1,2})\.(\d{1,2})\.(\d{2,4})', raw_date)
    if match_numeric:
        # Tries to return datetime object
        try:
            return pd.to_datetime(match_numeric.group(0), dayfirst=True, errors='coerce')
        except:
            pass

    
    # If both previous checks did not match, catch date strings in English format (e.g. January 11, 2019) with pattern:
    # Month name
    # Space
    # 1-2 digits
    # Comma
    # Space
    # 2-4 digits
    match_en = re.search(r'([a-zA-Z]+)\s+(\d{1,2}),\s+(\d{2,4})', raw_date)
    if match_en:
        try:
            m, d, y = match_en.groups()
            return pd.to_datetime(f"{d} {m} {y}", errors='coerce')
        except:
            pass

    return pd.NaT

Now the date is parsed using that function, and rows are dropped where no date could be parsed.

In [7]:
# --- PARSE DATES ---

# Dictionary to map German months
german_months = {
    'Januar': 'January', 'Februar': 'February', 'März': 'March', 'Mai': 'May',
    'Juni': 'June', 'Juli': 'July', 'Oktober': 'October', 'Dezember': 'December',
    # Also want to catch abbreviations
    'Jan': 'January', 'Feb': 'February', 'Mrz': 'March', 'Apr': 'April',
    'Jun': 'June', 'Jul': 'July', 'Aug': 'August', 'Sep': 'September', 
    'Okt': 'October', 'Nov': 'November', 'Dez': 'December'
}

# Apply parsing using the helper function
df['Date'] = df['Date_Raw'].progress_apply(parse_date)

# Drop rows where date could not be parsed
before_date_drop = len(df)
df = df.dropna(subset=['Date'])
print(f"Dropped {before_date_drop - len(df)} rows with unparseable dates.")

# Extract date only (remove time of day)
df['Date'] = df['Date'].dt.date

# Time filternsure we only keep articles within the relevant study period.
# This removes parsing errors (e.g., finding a birthdate "1990" in text)
START_DATE = pd.to_datetime("2015-11-01").date()
END_DATE = pd.to_datetime("2025-11-01").date()
df = df[(df['Date'] >= START_DATE) & (df['Date'] <= END_DATE)]

# Finally, drop the now unused Date_Raw column
df.drop("Date_Raw", axis='columns', inplace=True)

print(f"Dates parsed. Date range of the articles: {df['Date'].min()} to {df['Date'].max()}")
print(f"Valid articles in timeframe: {len(df)}")

  0%|          | 0/64008 [00:00<?, ?it/s]

Dropped 8744 rows with unparseable dates.
Dates parsed. Date range of the articles: 2015-11-01 to 2025-11-01
Valid articles in timeframe: 55263


Several artificial features used for the final filtering and the analysis are added to the DataFrame:
- *Word_Count*: This is useful to filter out snippets, image captions, or other short messages that are not articles and have not enough semantic substance to analyse sentiment.
- *Is_IT_Source*: A flag that distinguishes IT articles from public discourse.
- *Is_PR*: To filter out articles from Press Release agencies, as we only want to keep journalistic articles and not corporate communication. CURRENTLY INACTIVE, MIGHT BE READDED LATER
- *Year* and *Month*: Temporal features useful for aggregation and plots later.

In [8]:
# --- FEATURE ENGINEERING ---

# Word Count
df['Word_Count'] = df['Text'].str.split().str.len()

# Is IT Source (true if the source is in the IT source list)
pattern_it = '|'.join(IT_SOURCES)
df['Is_IT_Source'] = df['Source'].astype(str).str.contains(pattern_it, case=False, regex=True)

# Press Release (true if the source is a PR agency) MIGHT BE ADDED AGAIN LATER
pattern_pr = '|'.join(PR_SOURCES)
df['Is_PR'] = df['Source'].astype(str).str.contains(pattern_pr, case=False, regex=True)

# Year and Month
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month

Remove all press releases using the Is_PR flag, and remove short articles with less than 40 words.

In [9]:
# --- FINAL FILTERING ---

original_len = len(df)

# Remove Press Releases MIGHT BE READDED LATER
#df = df[df['Is_PR'] == False]

# Remove short snippets that are likely image captions, snippets, errors, or very short messages
MIN_WORD_COUNT = 40
df = df[df['Word_Count'] >= MIN_WORD_COUNT]

print(f"Final cleanup: {original_len} -> {len(df)} articles.")

Final cleanup: 55263 -> 54133 articles.


The final corpus is now saved to a csv file at *data/csv/corpus_final.csv*.

In [10]:
# --- SAVE CORPUS ---

# Save the DataFrame to CSV
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)
df.to_csv(OUTPUT_FILE, index=False)

print("="*60)
print(f"FINAL CORPUS CREATED: {OUTPUT_FILE}")
print("="*60)

FINAL CORPUS CREATED: ../data/csv/corpus.csv


Here is an overview of some basic statistics of the saved DataFrame.

In [11]:
# --- STATISTICS ---

print(f"Total Articles: {len(df)}")
print("-" * 40)
print("Distribution by Source Type:")
print(df['Is_IT_Source'].value_counts().rename({True: 'IT Articles', False: 'General Articles'}))
print("-" * 40)
print("Timeframe:")
print(f"Start: {df['Date'].min()}")
print(f"End:   {df['Date'].max()}")
print("-" * 40)
print("Average Word Count:", int(df['Word_Count'].mean()))

Total Articles: 54133
----------------------------------------
Distribution by Source Type:
Is_IT_Source
General Articles    47087
IT Articles          7046
Name: count, dtype: int64
----------------------------------------
Timeframe:
Start: 2015-11-01
End:   2025-11-01
----------------------------------------
Average Word Count: 714
